In [2]:
import requests #for webscraping
from bs4 import BeautifulSoup #for webscraping
from lxml import html #for parsing html
import pandas as pd
import numpy as np
import io


def get_product_links():

    ''' This function scrapes product reviews on the beer advocate website using BeautifulSoup'''

    df = pd.DataFrame(columns=['product_name', 'product_review', 'user_rating']) #create empty dataframe to store product reviews 
    
    base_url = 'https://www.beeradvocate.com'
    
    page = requests.get(base_url+'/beer/top-rated/')
    soup = BeautifulSoup(page.text,'html.parser')
    
    cells = soup.find_all('td')
    for cell in cells:
        if len(cell) == 2:            
            product = cell.find('b').text
            external_url = cell.find('a')['href']
            
            product_page = requests.get(base_url+external_url)
            tree = html.fromstring(product_page.content)
            for i in range(1,26):
                try:
                    rating = tree.xpath('/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/div[8]/div/div[{}]/div[2]/span[2]/text()'.format(i))[0]
                                         #/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/div[8]/div/div[1]/div[2]/span[2]
                    comments = tree.xpath('/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/div[8]/div/div[{}]/div[2]/text()'.format(i))[1:6]
                                         #/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/div[8]/div/div[1]/div[2]/text()[2]
                    comment = ""
                    for com in comments:
                        comment += com.strip('\n') + " "

                    df = df.append({'product_name': product,
                                    'product_review': comment, 
                                    'user_rating': rating}, 
                                    ignore_index=True)
                except:
                    pass

    return df 



In [3]:
beer_reviews = get_product_links() 
beer_reviews.head()

,product_name,product_review,user_rating
0,Kentucky Brunch Brand Stout,Smell: early morning pancakes and coffee befor...,5.00
1,Kentucky Brunch Brand Stout,2019 vintage. Pours a very dark brown color wi...,4.53
2,Kentucky Brunch Brand Stout,It's hyped... There is a lot of breweries doin...,1.49
3,Kentucky Brunch Brand Stout,Reviewing 2019 vintage. This pours thick and c...,4.52
4,Kentucky Brunch Brand Stout,2018 version. Poured dark with a small head. S...,4.99
